In [9]:
import quandl
import pandas as pd
import numpy as np
import talib
import csv
import warnings
import seaborn as sns
from hmmlearn.hmm import GaussianHMM
from matplotlib import cm, pyplot as plt
from matplotlib.dates import YearLocator, MonthLocator
from functools import total_ordering
from testing import test_model_mapping,series_merge
from trading import generate_signals, eval_signals,time_series_chart
from commodities import hard_tech,hard,hard_pct
from pandas.plotting import register_matplotlib_converters
from testing import *
register_matplotlib_converters()

In [3]:
df=hard_pct.join(hard_tech)['1993-01-02':]

In [6]:
hard_pct

,pct_gold,pct_oil,pct_silver,pct_ruth,pct_irid,pct_pall,pct_rhod,pct_plat
Date,,,,,,,,
1992-07-01,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
1992-07-02,0.002621,0.008219,0.001236,0.0,0.000000,0.012048,0.018692,0.010390
1992-07-03,0.005315,0.000000,0.002469,0.0,0.000000,0.000000,0.000000,-0.007712
1992-07-06,0.000347,-0.009511,0.000000,0.0,0.000000,0.000000,0.000000,0.005181
1992-07-07,0.000866,-0.016461,0.000000,0.0,0.000000,-0.011905,-0.009174,-0.012887
...,...,...,...,...,...,...,...,...
2020-04-21,0.007517,-1.240941,0.004610,0.0,0.009615,-0.135655,-0.063830,-0.065134
2020-04-22,-0.001022,0.530864,-0.025893,0.0,0.000000,0.045069,-0.079545,0.038251
2020-04-23,0.015939,0.104106,0.025908,0.0,0.000000,0.021816,0.012346,0.013158


In [10]:
def test_model_mapping(df,assets, model=GaussianHMM,n_components=3,train_size=5000,begin_date='2015-01-01'):
    #makes a Markov model using the df, and finds the optimal mapping from states to trade signals
    train=df.iloc[:train_size]
    hmm=model(n_components=3, covariance_type="full", n_iter=1000).fit(train)
    models=[]
    for mapping in [[x,y,z] for x in [-1,0,1] for y in [-1,0,1] for z in [-1,0,1]]:
        def strat(row,hmm=hmm,mapping=mapping):
            ret=hmm.predict([row])[0]
            return mapping[ret]
        trade=generate_signals(df.loc[begin_date:],strat)
        for asset in assets.columns:
            prices,sigs=series_merge(assets[asset][begin_date:],trade)
            perf=eval_signals(prices,sigs)
            models.append(MarkovStrategy(strat,perf,asset))
    return sorted(models,reverse=True)

In [13]:
gold=test_model_mapping(df,pd.DataFrame(hard_pct['pct_gold']))

In [22]:
best=pd.DataFrame()
for asset in hard_pct.columns:
    best[asset[asset.find('_')+1:]]=test_model_mapping(df,pd.DataFrame(hard_pct[asset]))[0].returns

In [23]:
best

,gold,oil,silver,ruth,irid,pall,rhod,plat
Date,,,,,,,,
2015-01-02,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
2015-01-05,1.005238,0.949355,1.008197,1.000000,1.000000,1.002513,1.000000,0.988294
2015-01-06,1.016960,0.910091,1.023644,0.965517,1.000000,1.006281,1.000000,0.983394
2015-01-07,1.024758,0.896624,1.007881,0.965517,1.000000,1.006281,0.979920,0.979350
2015-01-08,1.020695,0.898649,1.010021,0.965517,1.000000,0.999984,0.971888,0.976142
...,...,...,...,...,...,...,...,...
2020-04-21,1.425030,14.566998,1.220299,4.655172,2.916667,2.654843,7.068273,1.257283
2020-04-22,1.423573,6.833900,1.251896,4.655172,2.916667,2.654843,6.506024,1.209190
2020-04-23,1.446264,6.122453,1.219462,4.655172,2.916667,2.654843,6.586345,1.193280


In [24]:
best.to_csv('hmm_results.csv',index=True)